気になっていること
* PositiveとNegativeは実際どれくらい離れているのか
* 特徴量は縮退していないか
* 一部の次元だけで判断されていないか =>これはCMD的にも多分発生してはいなそうではある


手法アイディア
* ランダム行列を挟んだ上で、複数の重みでスコアを出す
* Maximum Classifier Discrepancyみたいなことをする
* Hard Negative以外は無視する（Positiveの最小スコアより低いNegativeについては信用しない） => 学習が遅くなりそう（初期で全く学習がされない？）

In [1]:
from run_sacred import data_ingredient, method_ingredient, optim_ingredient

In [2]:
from sacred import Experiment
ex = Experiment('jupyter_ex', ingredients=[data_ingredient, method_ingredient, optim_ingredient], interactive=True)

In [3]:
@ex.main
def main(_config):
    print(_config)
    return _config

In [4]:
ex.run(config_updates={'seed':123456})

WARNING - jupyter_ex - No observers have been added to this run
INFO - jupyter_ex - Running command 'main'
INFO - jupyter_ex - Started
INFO - jupyter_ex - Result: {'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}
INFO - jupyter_ex - Completed after 0:00:00


{'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}


In [5]:
_config = {'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}

In [6]:
from run_sacred import *
from sacred_wrap import MongoExtractor
from utils import flatten_dict
from copy import deepcopy
import os
import torch

In [7]:
datasets = get_dataset(**_config['dataset'])
train_dataset_joint, valid_dataset_joint, train_dataset_marginal, valid_dataset_marginal, test_dataset = datasets


NameError: name 'train_dataset' is not defined

In [8]:
# Load pretrained model
model = get_model(input_shape=train_dataset_joint.get('input_shape'), K=_config['dataset']['K'], **_config['method'])
query = deepcopy(_config)
query = flatten_dict(query)
extractor = MongoExtractor(None, 'CPC_test')
result = list(extractor.find(query, ['config', 'info'], False, 'COMPLETED'))
assert len(result) == 1, "There are too many or no results. Please check the query {}".format(query)
result = result[0]

"""DGX上ではここのコメントを外す

path = os.path.join(result['info']['log_dir'], 'model_{}.pth'.format(_config['optim']['num_batch']))
model = model.cpu()
model.load_state_dict(torch.load(path, map_location='cpu'))
model = model.cuda()
"""

1600 800


"DGX上ではここのコメントを外す\n\npath = os.path.join(result['info']['log_dir'], 'model_{}.pth'.format(_config['optim']['num_batch']))\nmodel = model.cpu()\nmodel.load_state_dict(torch.load(path, map_location='cpu'))\nmodel = model.cuda()\n"

In [9]:
model

CPCModel(
  (g_enc): Encoder(
    (feature): Sequential(
      (0): Conv2d(1, 50, kernel_size=(1, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(50, 40, kernel_size=(1, 5), stride=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(40, 20, kernel_size=(1, 3), stride=(1, 1))
      (7): ReLU()
      (8): Dropout(p=0.5)
      (9): Flatten()
      (10): Linear(in_features=4520, out_features=1600, bias=True)
      (11): ReLU()
      (12): Dropout(p=0.5)
    )
  )
  (c_enc): ContextEncoder(
    (gru): GRU(1600, 800)
  )
  (predictor): Predictor(
    (linears): ModuleList(
      (0): Linear(in_features=800, out_features=1600, bias=True)
      (1): Linear(in_features=800, out_features=1600, bias=True)
      (2): Linear(in_features=800, out_features=1600, bias=True)
      (3): Linear(in_features=800, out_f

In [10]:
train_loader_joint = data.DataLoader(
    train_dataset_joint, batch_size=_config['optim']['batch_size'], shuffle=True)
train_loader_marginal = data.DataLoader(
    train_dataset_marginal, batch_size=_config['optim']['batch_size'], shuffle=True)


In [11]:
optimizer = optim.Adam(model.parameters(), lr=_config['optim']['lr'])

In [15]:
for num_iter in range(100):
    loss = train_CPC(train_loader_joint, train_loader_marginal, model, optimizer)

In [22]:
train_result = validate(train_dataset_joint, train_dataset_marginal, model, num_eval=1)

RuntimeError: CUDA out of memory. Tried to allocate 71.75 MiB (GPU 0; 10.91 GiB total capacity; 9.46 GiB already allocated; 20.50 MiB free; 56.33 MiB cached)

In [21]:
def validate(dataset_joint, dataset_marginal, model, num_eval=10, batch_size=128):
    """Evaluate the model."""
    model.eval()

    loader_joint = data.DataLoader(dataset_joint, batch_size=batch_size, shuffle=True, drop_last=True)
    loader_marginal = data.DataLoader(dataset_marginal, batch_size=batch_size, shuffle=True, drop_last=True)

    if num_eval is None:
        num_eval = len(loader_joint)

    K = dataset_marginal.T
    L = dataset_joint.T - K
    losses = [0] * K
    TP = [0] * K
    TN = [0] * K
    FP = [0] * K
    FN = [0] * K

    for i, ((X_j, _), (X_m, _)) in enumerate(zip(loader_joint, loader_marginal)):
        X_j = X_j.float().cuda()
        X_m = X_m.float().cuda()

        score_j_list, score_m_list = model(X_j, X_m, L, K)
        for k in range(K):
            losses[k] += (-1.0 * torch.log(torch.clamp(score_j_list[k], min=1e-8)).mean()
                          - torch.log(torch.clamp(1-score_m_list[k], min=1e-8)).mean()).item()
            TP[k] += (score_j_list[k] > 0.5).sum().item()
            TN[k] += (score_m_list[k] < 0.5).sum().item()
            FP[k] += (score_m_list[k] > 0.5).sum().item()
            FN[k] += (score_j_list[k] < 0.5).sum().item()

        if i+1 == num_eval:
            break
    results = OrderedDict()
    print(score_j_list)
    for k in range(K):
        results['loss-{}'.format(k)] = losses[k] / (2*(i+1))
        results['accuracy-{}'.format(k)] = float(TP[k]+TN[k]) / float(FP[k]+FN[k]+TP[k]+TN[k])

    model.train()
    return results